In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
S = 25
alpha = 0.25
beta = 1.0
C = 0.15
T = 100
arrival_type = "sigmoid"
departure_type = "sigmoid"

In [ ]:
filename = f"S-{S}_alpha-{alpha}_beta-{beta}_C-{C}_T-{T}_{arrival_type}_{departure_type}"
adjacency_matrices = np.load(f"../results/{filename}_temporal_adjacencies.npy")

In [ ]:
adjacency_matrices.shape

In [ ]:
node_activity_intervals = dict()
for t in range(adjacency_matrices.shape[0]):
    adj = adjacency_matrices[t]
    G = nx.from_numpy_array(adj)
    active_node_degrees = []
    active_nodes = []
    for u in G:
        if G.degree(u) > 0:
            active_nodes.append(u)
            active_node_degrees.append(G.degree(u))
    for u in active_nodes:
        if u not in node_activity_intervals:
            node_activity_intervals[u] = [t+1, t+1]
        else:
            node_activity_intervals[u][1] += 1

    if len(active_node_degrees) > 0:
        print(len(active_nodes),
              G.number_of_edges(),
              min(active_node_degrees),
              np.mean(active_node_degrees),
              max(active_node_degrees))
    else:
        print(len(active_nodes),
              G.number_of_edges(), 0, 0, 0)

In [ ]:
filename = f"S-{S}_alpha-{alpha}_beta-{beta}_C-{C}_T-{T}_{arrival_type}_{departure_type}"
for arrival_type in ["uniform", "exponential", "sigmoid"]:
    for departure_type in ["uniform", "exponential", "sigmoid"]:
        plt.figure()
        filename = f"S-{S}_alpha-{alpha}_beta-{beta}_C-{C}_T-{T}_{arrival_type}_{departure_type}"
        species_mass = np.load(f"../results/{filename}_species_mass.npy")
        node_activities = np.load(f"../results/{filename}_node_activities.npy")
        for i in np.argsort(species_mass)[::-1]:
            nonz = np.nonzero(node_activities[:, i])[0]
            if nonz.shape[0] > 0:
                plt.plot((nonz.min(), nonz.max()), [i]*2, c="black", alpha=0.6)
            else:
                plt.plot((0, T), [i]*2, c="black", ls="--", alpha=0.1)
        plt.title(arrival_type + ", " + departure_type)
        plt.xlim(0, T)
        plt.ylim(-0.5, 25)
        plt.xlabel("Time")
        plt.ylabel("Species")
        plt.tight_layout()
        plt.savefig(f"../results/plots/{filename}_intervals.png", dpi=150, facecolor=None)

In [ ]:
filename = f"S-{S}_alpha-{alpha}_beta-{beta}_C-{C}_T-{T}_{arrival_type}_{departure_type}"
for arrival_type in ["uniform", "exponential", "sigmoid"]:
    for departure_type in ["uniform", "exponential", "sigmoid"]:
        plt.figure()
        filename = f"S-{S}_alpha-{alpha}_beta-{beta}_C-{C}_T-{T}_{arrival_type}_{departure_type}"
        full_adj = np.load(f"../results/{filename}_adj_matrix_full.npy")
        G = nx.from_numpy_array(full_adj)
        maximum_degree = max([G.degree(u) for u in G])
        total_edges = G.number_of_edges()
        adjacency_matrices = np.load(f"../results/{filename}_temporal_adjacencies.npy")
        node_activities = np.load(f"../results/{filename}_node_activities.npy")
        mean_degrees = []
        max_degrees = []
        num_edges = []
        for t in range(adjacency_matrices.shape[0]):
            adj = adjacency_matrices[t]
            G = nx.from_numpy_array(adj)
            active_node_degrees = []
            active_nodes = []
            for u in G:
                if node_activities[t, u] == 1:
                    active_nodes.append(u)
                    active_node_degrees.append(G.degree(u))
        
            if len(active_node_degrees) > 0:
                mean_degrees.append(np.mean(active_node_degrees) / maximum_degree)
                max_degrees.append(np.max(active_node_degrees) / maximum_degree)
                num_edges.append(G.number_of_edges() / total_edges)
            
        plt.title(arrival_type + ", " + departure_type + f" {total_edges} Edges, Max Degree: {maximum_degree}")
        plt.plot(list(range(len(mean_degrees))), mean_degrees, label="Mean Degree")
        plt.plot(list(range(len(max_degrees))), max_degrees, label="Max Degree")
        plt.plot(list(range(len(num_edges))), num_edges, label="Edges")
        plt.ylabel("Proportion of Full")
        plt.xlabel("Time")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"../results/plots/{filename}_degrees.png", dpi=150, facecolor=None)